In [1]:
using TropicalFrechetMeans
using CDDLib, Polyhedra
using Oscar

In [2]:
function trop_normalize(x)
    return x .- first(x)
end

function trop_normalize(x::Vector{T}) where {T<:TropicalSemiringElem}
    return x ./ first(x)
end

function indices(v)
    findfirst(==(-1), v), findfirst(==(1), v)
end

indices (generic function with 1 method)

In [3]:
sample = [[-3,0,0], [0,-6,0], [0,0,-12]]
n = length(sample |> first)

num_FM = tropical_frechet_mean(sample) |> trop_normalize
@show num_FM
P = tropical_frechet_set(sample; tol=1e-3)

num_FM = [0.0, -1.4387319974273893e-8, -1.0001294161599674]


Polyhedron CDDLib.Polyhedron{Rational{BigInt}}:
5-element iterator of HalfSpace{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HalfSpace(Rational{BigInt}[1, 0, -1], 1//1)
 HalfSpace(Rational{BigInt}[-1, 1, 0], 1//1)
 HalfSpace(Rational{BigInt}[0, 1, -1], 1//1)
 HalfSpace(Rational{BigInt}[-1, 0, 1], -1//1)
 HalfSpace(Rational{BigInt}[0, -1, 1], -1//1)

In [4]:
vrep(P)

V-representation CDDGeneratorMatrix{Rational{BigInt}, GMPRational}:
1-element iterator of Vector{Rational{BigInt}}:
 Rational{BigInt}[1, 1, 0],
1-element iterator of Line{Rational{BigInt}, Vector{Rational{BigInt}}}:
 Line(Rational{BigInt}[1, 1, 1])

In [5]:
H = hrep(P)

H-representation CDDInequalityMatrix{Rational{BigInt}, GMPRational}:
5-element iterator of HalfSpace{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HalfSpace(Rational{BigInt}[1, 0, -1], 1//1)
 HalfSpace(Rational{BigInt}[-1, 1, 0], 1//1)
 HalfSpace(Rational{BigInt}[0, 1, -1], 1//1)
 HalfSpace(Rational{BigInt}[-1, 0, 1], -1//1)
 HalfSpace(Rational{BigInt}[0, -1, 1], -1//1)

In [6]:
T = tropical_semiring()
C = identity_matrix(T, n)
for h in halfspaces(H)
    setindex!(C, h.β, indices(h.a)...)
end
Matrix(C)

3×3 Matrix{TropicalSemiringElem{typeof(min)}}:
 (0)  (1)  (-1)
 ∞    (0)  (-1)
 (1)  (1)  (0)

In [7]:
V = Set{Vector}()
A = C^n

for i in 1:size(A,1)
    for j in 1:size(A,1)
        if i != j
            for v in breakpoints_of_tropical_line(A[:,i], A[:,j])
                push!(V, trop_normalize(v))
            end
        end
    end
end

UndefVarError: UndefVarError: `breakpoints_of_tropical_line` not defined

## Phylogenetic trees

In [8]:
using JSON3
using DataFrames

# Function to read the JSON file and convert to a list of matrices
function read_and_convert_json(file_path::String)
    # Read the JSON file
    json_data = JSON3.read(file_path)
    
    # Extract elements from the nested arrays
    elements = [x[1] for x in json_data]
    elements = rationalize.(10000 * elements, tol=1e-2)
    
    # Convert elements into matrices
    num_elements = length(elements)
    matrices = []
    
    for i in 1:64:num_elements
        # Get the next 64 elements
        matrix_elements = elements[i:min(i+63, num_elements)]
        
        # Convert to an 8x8 matrix if there are 64 elements, otherwise create a smaller matrix
        matrix_size = length(matrix_elements)
        sqrt_size = Int(sqrt(matrix_size))
        push!(matrices, reshape(matrix_elements, sqrt_size, sqrt_size))
    end
    
    return matrices
end

"""
Take a matrix of pairwise distances between taxa and returns the cophenetic vector.
"""
function cophenetic_from_distance(pairwise)
    n = size(pairwise, 1)
    coph = [pairwise[i, j] for i in 1:n-1 for j in i+1:n]
    return coph
end

# Read and convert the JSON file
file_path = "all_matrices.json"
matrices = read_and_convert_json(file_path)
taxa = ["Tg", "Et", "Cp", "Ta", "Bb", "Tt", "Pv", "Pf"]

coph_vecs = [cophenetic_from_distance(mat) for mat in matrices]

268-element Vector{Vector{Rational{Int64}}}:
 [3784, 6626, 9906, 6521, 11778, 8750, 7661, 7601, 10881, 7496  …  4901, 15579, 12551, 11462, 12194, 9167, 8078, 14217, 13128, 1089]
 [3485, 8484, 9427, 8865, 25257, 9257, 10300, 8814, 9756, 9195  …  3645, 25010, 9010, 10054, 24449, 8449, 9492, 20033, 21076, 2163]
 [3570, 5592, 6890, 4591, 4849, 5136, 4872, 3905, 6062, 3763  …  6609, 6867, 7154, 6890, 2383, 4367, 4103, 4625, 4361, 274]
 [1696, 4297, 5665, 6408, 4431, 4218, 4248, 4473, 5841, 6584  …  9039, 7062, 6849, 6880, 5542, 6984, 7014, 5006, 5037, 557]
 [1864, 8173, 6779, 10056, 10843, 9010, 9319, 8433, 7039, 10315  …  11759, 12546, 10713, 11022, 15116, 13283, 13592, 12332, 12642, 2722]
 [3801, 11567, 5222, 5725, 5642, 8167, 7548, 12836, 6491, 6994  …  6129, 6046, 8571, 7952, 601, 7410, 6791, 7327, 6708, 4443]
 [751, 29706, 6877, 4376, 5857, 9999, 8966, 29899, 7070, 4569  …  6079, 7560, 11702, 10669, 3359, 8305, 7272, 9787, 8754, 5104]
 [4080, 8200, 9699, 9924, 11942, 9302, 9463, 7689, 

In [9]:
"""
Check if a distance matrix defines a phylogenetic tree
"""
function is_phylogenetic_tree(D)
    n = size(D, 1)
    
    # Check if the matrix is symmetric and non-negative
    for i in 1:n
        for j in i:n
            if D[i, j] != D[j, i] || D[i, j] < 0
                return false
            end
        end
    end

    # Check the four-point condition
    for i in 1:n-3
        for j in i+1:n-2
            for k in j+1:n-1
                for l in k+1:n
                    # Calculate distances
                    D_ij_kl = D[i, j] + D[k, l]
                    D_ik_jl = D[i, k] + D[j, l]
                    D_il_jk = D[i, l] + D[j, k]
                    
                    # Check the four-point condition
                    if !(D_ij_kl >= D_ik_jl && D_ij_kl >= D_il_jk) &&
                       !(D_ik_jl >= D_ij_kl && D_ik_jl >= D_il_jk) &&
                       !(D_il_jk >= D_ij_kl && D_il_jk >= D_ik_jl)
                        return false
                    end
                end
            end
        end
    end
    
    return true
end

is_phylogenetic_tree

In [10]:
@time phylo_frech = tropical_frechet_set(coph_vecs)
println(phylo_frech)

 11.631271 seconds (94.31 M allocations: 7.301 GiB, 13.63% gc time, 3.02% compilation time: 24% of which was recompilation)
HalfSpace(Rational{BigInt}[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1], 0//1) ∩ HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1192//1) ∩ HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 866//1) ∩ HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 824//1) ∩ HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 677//1) ∩ HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1023//1) ∩ HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0], 1023//1) ∩ HalfSpace(Rati

In [11]:
n = length(coph_vecs |> first)
H = hrep(phylo_frech)

H-representation CDDInequalityMatrix{Rational{BigInt}, GMPRational}:
300-element iterator of HalfSpace{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HalfSpace(Rational{BigInt}[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1], 0//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1192//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 866//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 824//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 677//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1023//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0], 1023//

In [12]:
T = tropical_semiring()
C = identity_matrix(T, n)
for h in halfspaces(H)
    setindex!(C, h.β, indices(h.a)...)
end
Matrix(C)

28×28 Matrix{TropicalSemiringElem{typeof(min)}}:
 (0)  ∞       ∞       ∞       ∞       …  ∞            ∞            (0)
 ∞    (0)     ∞       ∞       ∞          ∞            ∞            (0)
 ∞    ∞       (0)     ∞       ∞          (775)        (880)        (0)
 ∞    ∞       ∞       (0)     ∞          (2233)       ∞            (0)
 ∞    ∞       ∞       (2400)  (0)        ∞            ∞            (0)
 ∞    ∞       (1705)  (1418)  (2254)  …  ∞            ∞            ∞
 ∞    (1192)  (1229)  (942)   (1778)     ∞            ∞            (0)
 ∞    ∞       ∞       ∞       ∞          ∞            ∞            (0)
 ∞    ∞       ∞       (2282)  (2240)     (65867//33)  (65867//33)  (0)
 ∞    (866)   (1580)  (2575)  (1828)     ∞            ∞            (0)
 ⋮                                    ⋱  ⋮                         
 ∞    ∞       ∞       (2518)  ∞          (2284)       ∞            (0)
 ∞    (1023)  ∞       (1922)  (1370)  …  (2231)       ∞            (0)
 ∞    (1477)  (1734)  (1303)  (15

In [13]:
Matrix(C^n)

28×28 Matrix{TropicalSemiringElem{typeof(min)}}:
 (0)  (1617)  (2058)  (2741)  (2560)  …  (2549)       (2262)       (0)
 (0)  (0)     (2058)  (2741)  (2560)     (2549)       (2262)       (0)
 (0)  (1617)  (0)     (1826)  (2071)     (775)        (880)        (0)
 (0)  (1617)  (2058)  (0)     (2560)     (2233)       (2262)       (0)
 (0)  (1057)  (2058)  (2400)  (0)        (2214)       (2262)       (0)
 (0)  (1023)  (1705)  (1418)  (2254)  …  (2180)       (2262)       (0)
 (0)  (1192)  (1229)  (942)   (1778)     (2004)       (2109)       (0)
 (0)  (1617)  (2058)  (2741)  (2560)     (2549)       (2262)       (0)
 (0)  (1617)  (2058)  (2282)  (2240)     (65867//33)  (65867//33)  (0)
 (0)  (866)   (1580)  (2575)  (1828)     (2355)       (1753)       (0)
 ⋮                                    ⋱  ⋮                         
 (0)  (1617)  (2058)  (2518)  (2560)     (2284)       (2262)       (0)
 (0)  (1023)  (2058)  (1922)  (1370)  …  (2231)       (2262)       (0)
 (0)  (1477)  (1734)  (1303)  (

In [14]:
function breakpoints_of_tropical_line(p,q)
    r = q ./ p
    σ = sortperm(r)

    return map(1:length(r)) do k
        [q[σ[1:k]]..., r[σ[k]] .* p[σ[k+1:end]]...][σ]
    end
end

breakpoints_of_tropical_line (generic function with 1 method)

In [15]:
V = Set{Vector}()
A = C^n

for i in 1:size(A,1)
    for j in 1:size(A,1)
        if i != j
            for v in breakpoints_of_tropical_line(A[:,i], A[:,j])
                push!(V, trop_normalize(v))
            end
        end
    end
end
V

Set{Vector} with 9497 elements:
  TropicalSemiringElem{typeof(min)}[(0), (0), (-2635), (-862), (0), (-268), (0)…
  TropicalSemiringElem{typeof(min)}[(0), (-131), (-37984//33), (-175), (0), (-2…
  TropicalSemiringElem{typeof(min)}[(0), (-498), (-381), (-2165), (-415), (0), …
  TropicalSemiringElem{typeof(min)}[(0), (-412), (354), (354), (-206), (-240), …
  TropicalSemiringElem{typeof(min)}[(0), (870), (-493), (1190), (863), (1190), …
  TropicalSemiringElem{typeof(min)}[(0), (-521), (0), (-745), (0), (-241), (-10…
  TropicalSemiringElem{typeof(min)}[(0), (412), (412), (-1753), (412), (-523), …
  TropicalSemiringElem{typeof(min)}[(0), (0), (0), (0), (-55), (-437), (-1667),…
  TropicalSemiringElem{typeof(min)}[(0), (-73), (222), (-2275), (202), (222), (…
  TropicalSemiringElem{typeof(min)}[(0), (18844//33), (18844//33), (3070//33), …
  TropicalSemiringElem{typeof(min)}[(0), (923), (895), (58), (923), (923), (923…
  TropicalSemiringElem{typeof(min)}[(0), (460), (460), (-1705), (-72), (-360)

In [16]:
ratV = reduce(hcat, map(collect(V)) do v
    QQ.(v)[2:end]
end) |> transpose

9497×27 transpose(::Matrix{QQFieldElem}) with eltype QQFieldElem:
 0           -2635       -862        …  -444        0           0
 -131        -37984//33  -175           0           -690        -760
 -498        -381        -2165          -553        -62635//33  0
 -412        354         354            354         -240        -240
 870         -493        1190           1190        458         440
 -521        0           -745        …  -186        0           -278
 412         412         -1753          -49039//33  359         31
 0           0           0              0           0           0
 -73         222         -2275          -126        222         -594
 18844//33   18844//33   3070//33       2509//33    7195//33    -8513//33
 ⋮                                   ⋱              ⋮           
 84          292         326            -309        326         139
 -109        66          66             66          -751        -398
 -165        495         -19306//33  …  -477     

In [17]:
FMP = convex_hull(ratV)

Polyhedron in ambient dimension 27